# 4. Finding the best ML model with H2o

Earlier we tested the models through trial and error. Now, we do the same using __autoMachineLearning__ from __H2o__.

In [30]:
#Data management
import pandas as pd
import joblib

#autoMachineLearning
import h2o
from h2o.automl import H2OAutoML
from h2o.grid.grid_search import H2OGridSearch

In [31]:
train = pd.read_csv('preprocessed_df.csv',index_col=0) #Loading TRAIN data

In [32]:
train

,SeniorCitizen,tenure,MonthlyCharges,ExtraChargesEstimate,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,0,0.013889,0.115423,0.498387,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,0,0.472222,0.385075,0.435493,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,1,0
2,0,0.027778,0.354229,0.498992,1,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,1
3,0,0.625000,0.239303,0.414057,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
4,0,0.027778,0.521891,0.512162,0,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0.333333,0.662189,0.438315,1,1,1,1,0,0,...,1,0,1,0,1,0,0,0,1,0
7039,0,1.000000,0.845274,0.407674,0,1,1,1,0,0,...,1,0,1,0,1,0,1,0,0,0
7040,0,0.152778,0.112935,0.526408,0,1,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
7041,1,0.055556,0.558706,0.510482,1,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1


In [33]:
h2o.init() #To start h2o

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 48 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_Nicolas_ulxozq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.593 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [34]:
h2train = h2o.H2OFrame(train) #H20Frame is equal to a Pandas dataframe. We parse so it can be later trained.

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [35]:
cat_cols = train.columns.drop([ 'tenure', 'MonthlyCharges', 'ExtraChargesEstimate'])

for col in cat_cols:
    h2train[col] = h2train[col].asfactor()



In [36]:
x = list(train.columns)
x.remove('Churn_Yes')

y = "Churn_Yes"

print("X:", x)
print("y:", y)

X: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'ExtraChargesEstimate', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed chec

In [37]:
#TRAINING all the h20 models

automl = H2OAutoML(max_models=30,max_runtime_secs=3600, sort_metric='AUCPR',  balance_classes=True)
automl.train(x=x, y=y, training_frame=h2train)

AutoML progress: |█
14:44:21.893: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),13/30
# GBM base models (used / total),6/14
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),6/13
# DRF base models (used / total),0/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [38]:
#Showing the best performers

leader_board = automl.leaderboard
leader_board.head()

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_5_20230313_144421,0.664714,0.847433,0.413846,0.236398,0.366299,0.134175
StackedEnsemble_BestOfFamily_1_AutoML_5_20230313_144421,0.660965,0.846319,0.41528,0.231343,0.367202,0.134837
GBM_5_AutoML_5_20230313_144421,0.660172,0.842176,0.42121,0.24068,0.369765,0.136726
GBM_1_AutoML_5_20230313_144421,0.65952,0.844944,0.416707,0.232621,0.367837,0.135304
GBM_grid_1_AutoML_5_20230313_144421_model_6,0.658633,0.844245,0.417443,0.233448,0.368158,0.13554
GBM_grid_1_AutoML_5_20230313_144421_model_5,0.657497,0.84543,0.416395,0.235303,0.36769,0.135196
GBM_grid_1_AutoML_5_20230313_144421_model_9,0.654366,0.842573,0.421035,0.240864,0.369856,0.136793
GBM_3_AutoML_5_20230313_144421,0.651661,0.83874,0.427762,0.238477,0.372162,0.138505
GBM_2_AutoML_5_20230313_144421,0.649919,0.839674,0.425333,0.245566,0.371489,0.138004
GLM_1_AutoML_5_20230313_144421,0.649447,0.843307,0.418909,0.237624,0.368899,0.136087


### In the leaderboard ⬆️ there are two types of models: stacked ensamble models & single models. We'll select the best of each category (since the stacked ones cannot be exported).

## 4.1. The best model is a stacked ensamble

Stacked Ensamble models cannot be exported, so we'll just apply it here: 

predicted_price_h2_stacked = automl.leader.predict(h2test_stacked).as_data_frame() #PREDICTING the PRICES on the TEST dataset
predicted_price_h2_stacked #Result

#Adding the PRICE column
stacked_test["price"] = predicted_price_h2_stacked 

#Creating a DF with just the two columns we need
stacked_columns = ["id","price"]
stacked_submission = stacked_test["price"]
stacked_test = stacked_test[stacked_columns]
stacked_test.head()

stacked_test.to_csv("output/submissions/predicted_price_h2_stacked.csv") #Saving the CSV

## 4.2. Saving the best single model predicted by h20

#Loading the TEST dataset

single_test = pd.read_csv('output/clean_diamonds_test.csv') #Cleaned Test dataset from Kaggle
h2test_single = h2o.H2OFrame(single_test) #Conversion into a H20 frame to train
h2test_single.head() #preview

single_model = h2o.get_model(automl.leaderboard.as_data_frame()['model_id'][3]) #Saving the best NON-STACKED model

joblib.dump(single_model, 'output/trained_models/predicted_price_h2_single.sav') #Saving the model


#Another way to save it:
model_path = h2o.save_model(model=single_model, path="output/trained_models/predicted_price_h2_single_again", force=True)
saved_model = h2o.load_model(model_path)

predicted_price_h2_single = single_model.predict(h2test_single).as_data_frame() #PREDICTING the PRICES on the TEST dataset
predicted_price_h2_single #preview

#Adding the PRICE column
single_test["price"] = predicted_price_h2_single

#Creating a DF with just the two columns we need
single_columns = ["id","price"]
single_submission = single_test["price"]
single_test = single_test[single_columns]
single_test.head()

predicted_price_h2_single.to_csv("output/submissions/predicted_price_h2_single.csv") #Saving the CSV

## Now we have two different files to upload to Kaggle to see which one did better.


# We're done 🚀
